In [3]:
from minio import Minio
from dotenv import load_dotenv
import os

In [7]:
load_dotenv()

minio_client = Minio(
    endpoint=os.getenv("MINIO_ENDPOINT"),
    access_key=os.getenv("MINIO_ACCESS_KEY"), 
    secret_key=os.getenv("MINIO_SECRECT_KEY"), 
    secure=False 
)
bucket_name = os.getenv("MINIO_BUCKET_NAME")

In [20]:
list_objects = minio_client.list_objects(bucket_name, prefix='Portalnoticiasceara', recursive=True)

In [18]:
for obj in list_objects:
    print(f'Obj: ', obj.object_name)

Obj:  Portalnoticiasceara/2024-12-10/media/photo_5390.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5394.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5395.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5401.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5406.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5407.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5410.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5411.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5419.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5420.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5421.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5422.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5423.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5424.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5425.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5429.j

In [19]:
for obj in list_objects:
    response = minio_client.get_object(bucket_name=bucket_name, object_name=obj.object_name)
    data = response.read()
    print(data)
    break

NameError: name 'list_objects' is not defined